In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np


In [ ]:
from collections import deque

def BFS(board, start_row, start_col):
  level = [[0]*5 for i in range(5)]
  #Stay Up Right Down Left, UR, DR, DL, UL
  dr = [0,-1,0,1,0, -1,1,1,-1]
  dc = [0,0,1,0,-1,1,1,-1,-1]

  result = 0
  if board[start_row][start_col] == 1 or board[start_row][start_col] == 2:
    return 0

  queue = deque([])
  visited = [[False]*5 for i in range(5)]
  
  level[start_row][start_col] = 1
  visited[start_row][start_col] = True
  queue.append((start_row,start_col))

  while(len(queue) != 0):
    (row,col) = queue.popleft()
    #result = result + 1
    result = result + (5-level[row][col])*5

    for i in range(9):
      nr = row + dr[i]
      nc = col + dc[i]
      if nr < 0 or nr >= 5 or nc < 0 or nc >= 5:
        continue
      if board[nr][nc] == 1 or board[nr][nc] == 2: #여기수정
        continue
      if visited[nr][nc] == True:
        continue

      visited[nr][nc] = True
      level[nr][nc] = level[row][col] +1
      queue.append((nr,nc))
  return result


temp = [[0]*5 for i in range(5)]
temp[3][0] = 1
temp[4][0] = 1
temp[4][1] = 1
BFS(temp,1,1)


235

In [ ]:
import copy
from tabulate import tabulate
import random

class BingoEnv(Env):
  def __init__(self):
    self.round = 0
    self.action_space = Discrete(25)
    #self.observation_space = Box(low=0, high=2, shape=(15,5,5), dtype=np.int32)
    self.observation_space = Box(low = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]), high = np.array([2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,10]),dtype=np.int32)
 
    #self.state = np.zeros((15,5,5))
    self.state = np.zeros(26)
    
    self.first_rand_pos = random.choice([i for i in range(0,25) if i not in [6]])
    self.second_rand_pos = random.choice([i for i in range(0,25) if i not in [6,self.first_rand_pos]])
    self.state[self.first_rand_pos] = 1
    self.state[self.second_rand_pos] = 1



  def check_bingo(self, board, row,col):
    result_list = []

    result = 1
    for i in range(5):
      result = result * board[row][i]
      if result == 0:
        break
    if result > 0:
      result_list.append("ROW")
    
    result = 1
    for i in range(5):
      result = result * board[i][col]
      if result == 0:
        break
    if result > 0:
      result_list.append("COL")

    if row == col:
      result = 1
      result = result * board[0][0] * board[1][1] * board[2][2] * board[3][3] * board[4][4]
      if result > 0:
        result_list.append("LC")

    if row+col == 4:
      result = 1
      result = result * board[0][4] * board[1][3] * board[2][2] * board[3][1] * board[4][0]
      if result > 0 :
        result_list.append("RC")

    return result_list

  def step(self, action):
    #self.round = self.round+1
    #cur_board = self.state[self.round]
    #self.state[self.round] = copy.deepcopy(self.state[self.round-1])
    self.round = self.state[-1] + 1
    self.state = self.state[:-1].reshape(5,5)
    row = action // 5
    col = action % 5
    bingo_count = 0
    #canClick = False

#None Up Right Down Left
    dr = [0,-1,0,1,0]
    dc = [0,0,1,0,-1]
    before_num = [-1,-1,-1,-1,-1]
    for i in range(5):
      nr = row + dr[i]
      nc = col + dc[i]
      if nr < 0 or nr >= 5 or nc < 0 or nc >= 5:
        continue

      if self.state[nr][nc] == 2:
        before_num[i] = 2
        continue

      elif self.state[nr][nc] == 1:
        before_num[i] = 1
        self.state[nr][nc] = 0
        
      elif self.state[nr][nc] == 0:
        before_num[i] = 0
        self.state[nr][nc] = 1
      
    for i in range(5):
      nr = row + dr[i]
      nc = col + dc[i]
      if before_num[i] == 2:
        continue
      if nr < 0 or nr >= 5 or nc < 0 or nc >= 5:
        continue

      if self.state[nr][nc] != 0:
        isBingo = self.check_bingo(self.state,nr,nc)
        bingo_count = bingo_count + len(isBingo)

        for dir in isBingo:
          if dir == "ROW":
            for i in range(5):
              self.state[nr][i] = 2
          elif dir == "COL":
            for i in range(5):
              self.state[i][nc] = 2
          elif dir == "LC":
            self.state[0][0] = 2
            self.state[1][1] = 2
            self.state[2][2] = 2
            self.state[3][3] = 2
            self.state[4][4] = 2
          elif dir == "RC":
            self.state[0][4] = 2
            self.state[1][3] = 2
            self.state[2][2] = 2
            self.state[3][1] = 2
            self.state[4][0] = 2
    
    
    three_bingo = True
    #rest_block_score = BFS(self.state,1,1) / 100
    #rest_block_score = BFS(self.state,1,1) / 50
    rest_block_score = BFS(self.state,1,1) / 10

    if self.round % 3 == 0 and bingo_count == 0:
      three_bingo = False
    else:
      reward = rest_block_score
    #elif self.round % 3 == 0 and bingo_count > 0:
    #  reward = (2/bingo_count) + rest_block_score
    #elif self.round % 3 != 0 and bingo_count > 0:
    #  reward = -0.5 * bingo_count + rest_block_score
    #else:
    #  reward = rest_block_score

    board_sum = 0
    for i in range(5):
      for j in range(5):
        board_sum = board_sum + self.state[i][j]

    if (board_sum == 50) or self.round == 9 or three_bingo == False:
      done = True
      if self.round == 9 and three_bingo == True:
        reward = rest_block_score
      else:
        reward = -100
    else:
      done = False
    
    #if done == False:
    #  reward += 1
    
    info ={}
    self.state = np.append(self.state.reshape(25),self.round)
    return self.state, reward, done, info

  def render(self):
    print("ROUND : ",self.state[-1])
    print(tabulate(self.state[0:25].reshape(5,5)))

  def reset(self):
    self.state = np.zeros(26)
    
    self.first_rand_pos = random.choice([i for i in range(0,25) if i not in [6]])
    self.second_rand_pos = random.choice([i for i in range(0,25) if i not in [6,self.first_rand_pos]])
    self.state[self.first_rand_pos] = 1
    self.state[self.second_rand_pos] = 1
    return self.state


#t = BingoEnv()
#t.render()
#t.step(0)
#t.step(3)
#t.step(9)
#t.step(24)

In [ ]:
a= np.array([5,4,3])
a = a[-1] + 7
print(a)
np.zeros((1,5))

10


array([[0., 0., 0., 0., 0.]])

In [ ]:
env = BingoEnv()
episodes = 10
for episode in range(1,episodes+1):
  state = env.reset()
  done = False
  reward_sum = 0

  while not done:
    env.render()
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    reward_sum += reward

  env.render()
  print("Episode : {} reward_sum : {}".format(episode, reward_sum))

ROUND :  0.0
-  -  -  -  -
0  0  0  0  0
0  0  1  0  0
0  0  0  0  0
0  0  0  0  0
0  0  1  0  0
-  -  -  -  -
ROUND :  1.0
-  -  -  -  -
0  1  1  1  0
0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
0  0  1  0  0
-  -  -  -  -
ROUND :  2.0
-  -  -  -  -
0  1  1  1  0
0  0  0  0  0
0  0  0  0  0
0  0  1  0  0
0  1  0  1  0
-  -  -  -  -
ROUND :  3.0
-  -  -  -  -
0  1  1  0  0
0  0  1  1  1
0  0  0  1  0
0  0  1  0  0
0  1  0  1  0
-  -  -  -  -
Episode : 1 reward_sum : -59.5
ROUND :  0.0
-  -  -  -  -
0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
1  0  0  1  0
0  0  0  0  0
-  -  -  -  -
ROUND :  1.0
-  -  -  -  -
0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
1  1  0  1  0
1  1  1  0  0
-  -  -  -  -
ROUND :  2.0
-  -  -  -  -
0  0  0  0  0
0  0  0  0  0
0  0  1  0  0
1  0  1  0  0
1  1  0  0  0
-  -  -  -  -
ROUND :  3.0
-  -  -  -  -
0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
1  1  0  1  0
1  1  1  0  0
-  -  -  -  -
Episode : 2 reward_sum : -60.0
ROUND :  0.0
-  -  -  -  -
0  0  0  0  0
0  0  0  

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(states, actions):
  model = Sequential()
  model.add(Flatten(input_shape=(1,26)))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(actions,activation='linear'))
  return model

In [ ]:
del model

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n
model = build_model(states,actions)


In [ ]:
model.summary()
states

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 26)                0         
                                                                 
 dense_3 (Dense)             (None, 50)                1350      
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dense_5 (Dense)             (None, 25)                1275      
                                                                 
Total params: 5,175
Trainable params: 5,175
Non-trainable params: 0
_________________________________________________________________


(26,)

In [ ]:
!pip install keras-rl2

     |████████████████████████████████| 52 kB 473 kB/s 


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy

def build_agent(model, actions):
  policy = BoltzmannQPolicy()
  memory = SequentialMemory(limit=50000,window_length=1)
  dqn = DQNAgent(model= model, memory = memory, policy= policy, nb_actions=actions, nb_steps_warmup=10,target_model_update=1e-2)
  return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=0.001), metrics=["mse"])
#dqn.fit(env,nb_steps = 200000, visualize= False, verbose=1) #  더 학습시킬 때는 아래 있는 코드를 이용해라!!!!!

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -100.000, steps: 3
Episode 2: reward: -100.000, steps: 3
Episode 3: reward: -73.500, steps: 3
Episode 4: reward: -80.500, steps: 3
Episode 5: reward: -100.000, steps: 3
Episode 6: reward: -79.000, steps: 3
Episode 7: reward: -100.000, steps: 3
Episode 8: reward: -56.000, steps: 3
Episode 9: reward: -100.000, steps: 3
Episode 10: reward: -100.000, steps: 3
Episode 11: reward: -79.000, steps: 3
Episode 12: reward: -82.000, steps: 6
Episode 13: reward: -61.000, steps: 3
Episode 14: reward: -65.000, steps: 3
Episode 15: reward: -78.500, steps: 3
Episode 16: reward: -100.000, steps: 3
Episode 17: reward: -65.000, steps: 3
Episode 18: reward: -65.000, steps: 6
Episode 19: reward: -79.000, steps: 3
Episode 20: reward: -82.500, steps: 3
Episode 21: reward: -100.000, steps: 3
Episode 22: reward: -61.000, steps: 3
Episode 23: reward: -79.000, steps: 3
Episode 24: reward: -80.500, steps: 3
Episode 25: reward: -56.000, steps: 3
Episode 26: reward: -6

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 40: reward: -65.000, steps: 3
Episode 41: reward: -78.000, steps: 3
Episode 42: reward: -62.000, steps: 3
Episode 43: reward: -78.500, steps: 3
Episode 44: reward: -82.500, steps: 3
Episode 45: reward: -63.000, steps: 3
Episode 46: reward: -62.500, steps: 6
Episode 47: reward: -100.000, steps: 3
Episode 48: reward: -62.000, steps: 3
Episode 49: reward: -56.000, steps: 3
Episode 50: reward: -100.000, steps: 3
Episode 51: reward: -63.500, steps: 3
Episode 52: reward: -59.000, steps: 6
Episode 53: reward: -82.500, steps: 3
Episode 54: reward: -81.000, steps: 3
Episode 55: reward: -78.500, steps: 3
Episode 56: reward: -80.500, steps: 3
Episode 57: reward: -100.000, steps: 3
Episode 58: reward: -62.000, steps: 3
Episode 59: reward: -82.000, steps: 3
Episode 60: reward: -62.500, steps: 3
Episode 61: reward: -100.000, steps: 3
Episode 62: reward: -38.500, steps: 6
Episode 63: reward: -82.000, steps: 3
Episode 64: reward: -79.000, steps: 3
Episode 65: reward: -59.000, steps: 6
Episode 

In [ ]:
model.save('9round_final.h5')

In [ ]:
dqn.fit(env,nb_steps = 200000, visualize= False, verbose=1)

Training for 200000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 16.4981
1116 episodes - episode_reward: 147.816 [-57.500, 171.500] - loss: 7.388 - mse: 77730.238 - mean_q: 74.653

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 16.4780
1113 episodes - episode_reward: 147.977 [-56.500, 171.500] - loss: 6.713 - mse: 79760.023 - mean_q: 74.747

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 16.6142
1113 episodes - episode_reward: 149.366 [-8.500, 171.500] - loss: 4.684 - mse: 80708.047 - mean_q: 74.740

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 16.5796
1113 episodes - episode_reward: 148.891 [-55.000, 171.500] - loss: 4.320 - mse: 79934.711 - mean_q: 74.891

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 84s 8ms/s

-4.5

In [ ]:
class TestBingoEnv(Env):
  def __init__(self,state):
    self.round = state[-1]
    self.action_space = Discrete(25)
    self.observation_space = Box(low = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]), high = np.array([2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,10]),dtype=np.int32)
    self.state = state



  def check_bingo(self, board, row,col):
    result_list = []

    result = 1
    for i in range(5):
      result = result * board[row][i]
      if result == 0:
        break
    if result > 0:
      result_list.append("ROW")
    
    result = 1
    for i in range(5):
      result = result * board[i][col]
      if result == 0:
        break
    if result > 0:
      result_list.append("COL")

    if row == col:
      result = 1
      result = result * board[0][0] * board[1][1] * board[2][2] * board[3][3] * board[4][4]
      if result > 0:
        result_list.append("LC")

    if row+col == 4:
      result = 1
      result = result * board[0][4] * board[1][3] * board[2][2] * board[3][1] * board[4][0]
      if result > 0 :
        result_list.append("RC")

    return result_list

  def step(self, action):
    #self.round = self.round+1
    #cur_board = self.state[self.round]
    #self.state[self.round] = copy.deepcopy(self.state[self.round-1])
    self.round = self.state[-1] + 1
    self.state = self.state[:-1].reshape(5,5)
    row = action // 5
    col = action % 5
    bingo_count = 0
    canClick = False

#None Up Right Down Left
    dr = [0,-1,0,1,0]
    dc = [0,0,1,0,-1]
    before_num = [-1,-1,-1,-1,-1]
    for i in range(5):
      nr = row + dr[i]
      nc = col + dc[i]
      if nr < 0 or nr >= 5 or nc < 0 or nc >= 5:
        continue

      if self.state[nr][nc] == 2:
        before_num[i] = 2
        continue

      elif self.state[nr][nc] == 1:
        before_num[i] = 1
        self.state[nr][nc] = 0
        
      elif self.state[nr][nc] == 0:
        before_num[i] = 0
        self.state[nr][nc] = 1
      
    for i in range(5):
      nr = row + dr[i]
      nc = col + dc[i]
      if before_num[i] == 2:
        continue
      if nr < 0 or nr >= 5 or nc < 0 or nc >= 5:
        continue

      if self.state[nr][nc] != 0:
        isBingo = self.check_bingo(self.state,nr,nc)
        bingo_count = bingo_count + len(isBingo)

        for dir in isBingo:
          if dir == "ROW":
            for i in range(5):
              self.state[nr][i] = 2
          elif dir == "COL":
            for i in range(5):
              self.state[i][nc] = 2
          elif dir == "LC":
            self.state[0][0] = 2
            self.state[1][1] = 2
            self.state[2][2] = 2
            self.state[3][3] = 2
            self.state[4][4] = 2
          elif dir == "RC":
            self.state[0][4] = 2
            self.state[1][3] = 2
            self.state[2][2] = 2
            self.state[3][1] = 2
            self.state[4][0] = 2
        
    
    three_bingo = True
    #rest_block_score = BFS(self.state,1,1) / 100
    #rest_block_score = BFS(self.state,1,1) / 50
    rest_block_score = BFS(self.state,1,1) / 10

    if self.round % 3 == 0 and bingo_count == 0:
      three_bingo = False
    else:
      reward = rest_block_score
    #elif self.round % 3 == 0 and bingo_count > 0:
    #  reward = (2/bingo_count) + rest_block_score
    #elif self.round % 3 != 0 and bingo_count > 0:
    #  reward = -0.5 * bingo_count + rest_block_score
    #else:
    #  reward = rest_block_score

    board_sum = 0
    for i in range(5):
      for j in range(5):
        board_sum = board_sum + self.state[i][j]

    if (board_sum == 50) or self.round == 9 or three_bingo == False:
      done = True
      if self.round == 9 and three_bingo == True:
        reward = rest_block_score
      else:
        reward = -100
    else:
      done = False
    
    #if done == False:
    #  reward += 1

    info ={}
    self.state = np.append(self.state.reshape(25),self.round)
    return self.state, reward, done, info

  def render(self):
    print("ROUND : ",self.state[-1])
    print(tabulate(self.state[0:25].reshape(5,5)))

  def reset(self):
    self.state = np.zeros(26)
    
    self.first_rand_pos = random.choice([i for i in range(0,25) if i not in [6]])
    self.second_rand_pos = random.choice([i for i in range(0,25) if i not in [6,self.first_rand_pos]])
    self.state[self.first_rand_pos] = 1
    self.state[self.second_rand_pos] = 1
    return self.state

In [ ]:

test_state = np.zeros(26)
test_first_rand_pos = random.choice([i for i in range(0,25) if i not in [6]])
test_second_rand_pos = random.choice([i for i in range(0,25) if i not in [6,test_first_rand_pos]])
test_state[test_first_rand_pos] = 1
test_state[test_second_rand_pos] = 1

t = TestBingoEnv(test_state)
t.render()
#tt = TestBingoEnv(np.array([1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,2]))
#tt.render()
#tt_state,_,_,_ = tt.step(6)
#print(tt_state[0:25].reshape(5,5))

ROUND :  0.0
-  -  -  -  -
0  0  0  0  0
0  0  0  1  0
0  0  0  0  0
0  0  1  0  0
0  0  0  0  0
-  -  -  -  -


In [ ]:
#반복
print("before")
print(t.state[0:25].reshape(5,5))
cell_num = np.argmax(model.predict(t.state.reshape(1,1,26)))
temp_state,tmp_reward,tmp_done,_ = t.step(cell_num)

print('ROUND : ',temp_state[-1])
print("action : ",cell_num //5 , cell_num %5)
print(temp_state[0:25].reshape(5,5))
print(tmp_reward,tmp_done)
print("-------------------------------")

before
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 2. 2.]
 [0. 0. 2. 0. 2.]
 [0. 2. 0. 1. 2.]
 [2. 2. 2. 2. 2.]]
ROUND :  9.0
action :  3 1
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 2. 2.]
 [0. 1. 2. 0. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]]
13.0 True
-------------------------------


In [ ]:
print(BFS([[1, 1, 0, 0, 2],
 [1, 0, 1, 2, 2],
 [0, 0, 2, 1, 2],
 [2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2],],1,1))

print(BFS([[0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 1, 1],
 [1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0]],1,1))

75
195


In [ ]:
tmp_actions = model.predict(np.array([0, 0, 0, 0, 0,
 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0,
 1, 0, 0, 0, 0,
 0, 1, 0, 0, 0, 0]).reshape(1,1,26))
print(tmp_actions)
tmp_num = np.argmax(tmp_actions)
print(tmp_num//5,tmp_num%5)

[[ -81.620514  -94.453835  -81.42285   -80.62964   -80.356995  -94.5878
   -98.66935   -95.862755  -81.02263   -80.79095   -79.92034   -90.25046
   -81.07934   -80.87428   -80.09678   -79.64154   -79.9207    -79.938736
   -88.60145   -83.07046   -79.84846   -80.0151    -93.35611  -107.32624
   -96.846436]]
3 0


In [ ]:
#del model
import tensorflow as tf
model = tf.keras.models.load_model('inc_gap_9round_final.h5')

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=["mae"])

In [ ]:
with tf.device('/device:GPU:0'):
  dqn.fit(env,nb_steps = 50000, visualize= False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: -8.1934
864 episodes - episode_reward: -94.836 [-98.760, -91.420] - loss: 0.353 - mae: 93.244 - mean_q: -89.133

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 100s 10ms/step - reward: -8.2511
870 episodes - episode_reward: -94.843 [-98.720, -91.380] - loss: 0.350 - mae: 93.155 - mean_q: -89.172

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 101s 10ms/step - reward: -8.2038
865 episodes - episode_reward: -94.836 [-98.580, -90.820] - loss: 0.361 - mae: 93.207 - mean_q: -89.177

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 101s 10ms/step - reward: -8.2109
866 episodes - episode_reward: -94.818 [-98.660, -91.400] - loss: 0.358 - mae: 93.187 - mean_q: -89.145

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 101s 10ms/step

In [ ]:
model.save("inc_gap_9round_final.h5")

reward 수정

In [ ]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())


import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
tf.debugging.set_log_device_placement(True)
with tf.device('/GPU:0'):
  gpu_model = Sequential()
  gpu_model.add(Flatten(input_shape=(1,26)))
  gpu_model.add(Dense(50, activation='relu'))
  gpu_model.add(Dense(50, activation='relu'))
  gpu_model.add(Dense(actions,activation='linear'))
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

  c = tf.matmul(a, b)
  print(c)  
  gpu_model.summary()
  print(gpu_model)
  
  gpu_policy = BoltzmannQPolicy()
  gpu_memory = SequentialMemory(limit=50000,window_length=1)
  gpu_dqn = DQNAgent(model= gpu_model, memory = gpu_memory, policy= gpu_policy, nb_actions=actions, nb_steps_warmup=10,target_model_update=1e-2)
  gpu_dqn.compile(Adam(learning_rate=1e-3), metrics=["mae"])

  gpu_dqn.fit(env,nb_steps = 10000, visualize= False, verbose=1)

  

Tensor("MatMul_7:0", shape=(2, 2), dtype=float32, device=/device:GPU:0)
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 26)                0         
                                                                 
 dense_21 (Dense)            (None, 50)                1350      
                                                                 
 dense_22 (Dense)            (None, 50)                2550      
                                                                 
 dense_23 (Dense)            (None, 25)                1275      
                                                                 
Total params: 5,175
Trainable params: 5,175
Non-trainable params: 0
_________________________________________________________________
Training for 10000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 43:10 - reward: 0.7000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 3902/10000 [==========>...................] - ETA: 1:44 - reward: -30.2383done, took 67.355 seconds


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10666592694238258993
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7164466880925710482
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]
